In [125]:
# data libaries
from tslearn.metrics import dtw
import numpy as np
import pandas as pd
import math

In [126]:
from datetime import datetime
import json

## Load Trajectory Data

In [127]:
with open("./search_p.json", 'r') as file:
    data_passenger = json.load(file)

In [128]:
loca_p = data_passenger["route"]["shape"]["coordinates"]
time_p = data_passenger["route"]["timestamps"]
data_p = []

In [129]:
for coordinates, timestamp in zip(loca_p, time_p):
    data_p.append([*coordinates, timestamp['$date']])

In [130]:
with open("./search_d.json", 'r') as file:
    data_driver = json.load(file)

In [131]:
loca_d = data_driver["route"]["shape"]["coordinates"]
time_d = data_driver["route"]["timestamps"]
data_d = []

In [132]:
for coordinates, timestamp in zip(loca_d, time_d):
    data_d.append([*coordinates, timestamp['$date']])

##  Data conversion

In [133]:
def convert_to_unix(timestamp) -> float:
    datetime_obj = datetime.fromisoformat(timestamp.replace('Z', '+00:00'))
    unix_time = datetime_obj.timestamp()
    return unix_time

In [134]:
def convert_data(matrix):
    # Convert the timestamps to Unix time using pandas (you could also use datetime module in Python)
    timestamps = [convert_to_unix(date) for _ , _ , date in matrix]

    # Create a numpy array for latitudes, longitudes, and Unix timestamps
    latitudes = np.array([lat for lat, _, _ in matrix])
    longitudes = np.array([lng for _, lng, _ in matrix])
    timestamps = np.array(timestamps)

    return np.array(list(zip(latitudes, longitudes, timestamps)))

In [135]:
np_passenger = convert_data(data_p)
np_driver = convert_data(data_d)

In [136]:
np_driver

array([[1.13569500e+01, 5.54078300e+01, 1.71500697e+09],
       [1.13627800e+01, 5.54085400e+01, 1.71500701e+09],
       [1.13635500e+01, 5.54065400e+01, 1.71500705e+09],
       [1.13629000e+01, 5.54063900e+01, 1.71500706e+09],
       [1.13614700e+01, 5.54003400e+01, 1.71500714e+09],
       [1.13592400e+01, 5.53989100e+01, 1.71500718e+09],
       [1.13635500e+01, 5.53920900e+01, 1.71500725e+09],
       [1.13594200e+01, 5.53907000e+01, 1.71500729e+09],
       [1.13564900e+01, 5.53886200e+01, 1.71500736e+09],
       [1.13418200e+01, 5.53857300e+01, 1.71500738e+09],
       [1.13134700e+01, 5.53777500e+01, 1.71500745e+09],
       [1.12957000e+01, 5.53702800e+01, 1.71500751e+09],
       [1.12871100e+01, 5.53677000e+01, 1.71500767e+09],
       [1.12247700e+01, 5.53625800e+01, 1.71500771e+09],
       [1.11867900e+01, 5.53624400e+01, 1.71500775e+09],
       [1.11688700e+01, 5.53608100e+01, 1.71500780e+09],
       [1.11605200e+01, 5.53590100e+01, 1.71500786e+09],
       [1.11431700e+01, 5.53514

### GPS point interpolation

In [137]:
# Calculate the distance between two points on the Earth's surface using Haversine formula.
def haversine_distance(lat1, lon1, lat2, lon2):
    R = 6371  # Radius of the Earth in kilometers
    d_lat = math.radians(lat2 - lat1)
    d_lon = math.radians(lon2 - lon1)
    a = math.sin(d_lat/2) * math.sin(d_lat/2) + math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) * math.sin(d_lon/2) * math.sin(d_lon/2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    d = R * c
    return d

In [138]:
# used to extend line segments for finer control.
def intermediate_points(lat1, lon1, lat2, lon2, n):
    intermediate_points = []
    total_distance = haversine_distance(lat1, lon1, lat2, lon2)
    segment_distance = total_distance / (n + 1)  # n+2 points including the start and end points
    for i in range(1, n + 1):
        f = segment_distance * i / total_distance
        lat = lat1 + f * (lat2 - lat1)
        lon = lon1 + f * (lon2 - lon1)
        intermediate_points.append((lat, lon))
    return intermediate_points

In [139]:
# Find the closest pair of points between two sets of points.
def closest_pair(points1, points2):
    min_distance = float('inf')
    closest_point_pair = None
    for p1 in points1:
        for p2 in points2:
            d = haversine_distance(p1[0], p1[1], p2[0], p2[1])
            if d < min_distance:
                min_distance = d
                closest_point_pair = (p1, p2)
    return closest_point_pair, min_distance

In [140]:
# p1_points = intermediate_points(p1_start[0], p1_start[1], p1_end[0], p1_end[1], 10)
distance = dtw(np_passenger, np_driver)

In [141]:
print(f"Distance from passenger to driver: {distance}")

Distance from passenger to driver: 0.0
